In [222]:

# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# import necessary modules
import joblib
import json
import os


In [223]:
# 1) Load the dataset
CSV_PATH = "./Dataset/Bitcoin - Dataset.csv"
df = pd.read_csv(CSV_PATH)
print("Dataset loaded successfully.")

Dataset loaded successfully.


In [224]:
# === INITIAL SNAPSHOT ===
print("\n=== INITIAL HEAD ===")
print(df.head())


=== INITIAL HEAD ===
         Date        Open        High         Low       Close   Adj Close  \
0  2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
1  2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2  2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
3  2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
4  2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

     Volume  
0  21056800  
1  34483200  
2  37919700  
3  36863600  
4  26580100  


In [225]:
# INITIAL INFO
print("\n=== INITIAL INFO ===")
print(df.info())



=== INITIAL INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2713 non-null   object 
 1   Open       2713 non-null   float64
 2   High       2713 non-null   float64
 3   Low        2713 non-null   float64
 4   Close      2713 non-null   float64
 5   Adj Close  2713 non-null   float64
 6   Volume     2713 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 148.5+ KB
None


In [226]:
# INITIAL MISSING VALUES
print("\n=== INITIAL MISSING VALUES ===")
print(df.isnull().sum())



=== INITIAL MISSING VALUES ===
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [227]:
# INITIAL DESCRIBE
print("\n=== INITIAL DESCRIBE ===")
print(df.describe())


=== INITIAL DESCRIBE ===
               Open          High           Low         Close     Adj Close  \
count   2713.000000   2713.000000   2713.000000   2713.000000   2713.000000   
mean   11311.041069  11614.292482  10975.555057  11323.914637  11323.914637   
std    16106.428891  16537.390649  15608.572560  16110.365010  16110.365010   
min      176.897003    211.731003    171.509995    178.102997    178.102997   
25%      606.396973    609.260986    604.109985    606.718994    606.718994   
50%     6301.569824   6434.617676   6214.220215   6317.609863   6317.609863   
75%    10452.399414  10762.644531  10202.387695  10462.259766  10462.259766   
max    67549.734375  68789.625000  66382.062500  67566.828125  67566.828125   

             Volume  
count  2.713000e+03  
mean   1.470462e+10  
std    2.001627e+10  
min    5.914570e+06  
25%    7.991080e+07  
50%    5.098183e+09  
75%    2.456992e+10  
max    3.509679e+11  


In [228]:
# 2) Clean target formatting
df["Close"] = df["Close"].astype(float)
print("\nTarget 'Close' column cleaned and converted to float.")



Target 'Close' column cleaned and converted to float.


In [229]:
# 3) Fix categorical issues BEFORE imputation
df["Date"] = pd.to_datetime(df["Date"])
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
print("\nDate column cleaned.")


Date column cleaned.


In [230]:
# 4) Impute missing values
df["Date"] = df["Date"].fillna(method='ffill')  # forward fill missing values
df["Open"].fillna(df["Open"].mean(), inplace=True)
df["High"].fillna(df["High"].mean(), inplace=True)
df["Low"].fillna(df["Low"].mean(), inplace=True)
df["Close"].fillna(df["Close"].mean(), inplace=True)
df["Adj Close"].fillna(df["Adj Close"].mean(), inplace=True)
df["Volume"].fillna(df["Volume"].mean(), inplace=True)
print("\nMissing values imputed successfully.")


Missing values imputed successfully.


C:\Users\ALI KEY\AppData\Local\Temp\ipykernel_15340\3518086508.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Date"] = df["Date"].fillna(method='ffill')  # forward fill missing values


In [231]:
# 5) Remove duplicates
Before = df.shape
df = df.drop_duplicates()
After = df.shape
print(f"Dropped duplicates: {Before} → {After}")

Dropped duplicates: (2713, 7) → (2713, 7)


In [232]:
# 6) IQR Capping (no leakage)

# Define function to calculate lower and upper bounds using IQR
def iqr_fun(series, k=1.5):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    return lower, upper

# Apply IQR function to Bitcoin price-related columns
low_open, high_open = iqr_fun(df["Open"])
low_high, high_high = iqr_fun(df["High"])
low_low, high_low = iqr_fun(df["Low"])
low_close, high_close = iqr_fun(df["Close"])
low_volume, high_volume = iqr_fun(df["Volume"])

# Cap outliers within calculated bounds
df["Open"] = df["Open"].clip(lower=low_open, upper=high_open)
df["High"] = df["High"].clip(lower=low_high, upper=high_high)
df["Low"] = df["Low"].clip(lower=low_low, upper=high_low)
df["Close"] = df["Close"].clip(lower=low_close, upper=high_close)
df["Volume"] = df["Volume"].clip(lower=low_volume, upper=high_volume)

print("\n✅ Outliers capped successfully using IQR method.")



✅ Outliers capped successfully using IQR method.


In [233]:
# 7) One-Hot Encode Categorical Features

# First, make sure 'Date' is datetime
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Extract meaningful features from Date
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["DayOfWeek"] = df["Date"].dt.dayofweek

# One-hot encode Month and DayOfWeek
categorical_cols = ["Month", "DayOfWeek"]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print("\n✅ Categorical features one-hot encoded successfully.")



✅ Categorical features one-hot encoded successfully.


In [234]:
# 8) Feature engineering (no leakage)
df["Open-Close"] = df["Open"] - df["Close"]
df["High-Low"] = df["High"] - df["Low"]
df["Low-Close"] = df["Low"] - df["Close"]
print("\nNew features created successfully.")


New features created successfully.


In [235]:
# 9) Feature scaling (X only; keep targets & dummies unscaled)
scaler = StandardScaler()
df[["Open", "High", "Low", "Close", "Volume", 
    "Open-Close", "High-Low", "Low-Close"]] = scaler.fit_transform(df[["Open", "High", "Low", 
    "Close", "Volume", "Open-Close", "High-Low", "Low-Close"
    ]])

print("\n✅ Features scaled successfully.")


✅ Features scaled successfully.


In [236]:
# Save the scaler and the training feature order (X columns) for later use

# Create 'models' folder if it doesn't exist
os.makedirs("models", exist_ok=True)

# Save the scaler
joblib.dump(scaler, "models/bitcoin_scaler.pkl")

# Determine the target columns actually in the DataFrame
targets = [col for col in ["Close", "Log_Close"] if col in df.columns]

# Save the training feature columns (all except targets)
TRAIN_COLUMNS = df.drop(columns=targets).columns.tolist()
json.dump(TRAIN_COLUMNS, open("models/train_columns.json", "w"))

print("\n✅ Scaler and training columns saved successfully!")


✅ Scaler and training columns saved successfully!


In [237]:
# # === FINAL SNAPSHOT ===
print("\n=== FINAL HEAD ===")
print(df.head())


=== FINAL HEAD ===
        Date      Open      High       Low     Close   Adj Close    Volume  \
0 2014-09-17 -0.885645 -0.884096 -0.886190 -0.886821  457.334015 -0.806132   
1 2014-09-18 -0.886700 -0.885381 -0.890919 -0.890669  424.440002 -0.805367   
2 2014-09-19 -0.890539 -0.888680 -0.894355 -0.894138  394.795990 -0.805171   
3 2014-09-20 -0.893987 -0.889196 -0.893711 -0.892487  408.903992 -0.805232   
4 2014-09-21 -0.892416 -0.890431 -0.893315 -0.893667  398.821014 -0.805817   

   Year  Month_2  Month_3  ...  Month_12  DayOfWeek_1  DayOfWeek_2  \
0  2014    False    False  ...     False        False         True   
1  2014    False    False  ...     False        False        False   
2  2014    False    False  ...     False        False        False   
3  2014    False    False  ...     False        False        False   
4  2014    False    False  ...     False        False        False   

   DayOfWeek_3  DayOfWeek_4  DayOfWeek_5  DayOfWeek_6  Open-Close  High-Low  \
0        Fa

In [238]:
# === FINAL DESCRIBE ===
print("\n=== FINAL INFO ===")
print(df.info())


=== FINAL INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2713 entries, 0 to 2712
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         2713 non-null   datetime64[ns]
 1   Open         2713 non-null   float64       
 2   High         2713 non-null   float64       
 3   Low          2713 non-null   float64       
 4   Close        2713 non-null   float64       
 5   Adj Close    2713 non-null   float64       
 6   Volume       2713 non-null   float64       
 7   Year         2713 non-null   int32         
 8   Month_2      2713 non-null   bool          
 9   Month_3      2713 non-null   bool          
 10  Month_4      2713 non-null   bool          
 11  Month_5      2713 non-null   bool          
 12  Month_6      2713 non-null   bool          
 13  Month_7      2713 non-null   bool          
 14  Month_8      2713 non-null   bool          
 15  Month_9      2713 non-null   bool  

In [239]:
# === FINAL MISSING VALUES ===
print("\n=== FINAL MISSING VALUES ===")
print(df.isnull().sum())


=== FINAL MISSING VALUES ===
Date           0
Open           0
High           0
Low            0
Close          0
Adj Close      0
Volume         0
Year           0
Month_2        0
Month_3        0
Month_4        0
Month_5        0
Month_6        0
Month_7        0
Month_8        0
Month_9        0
Month_10       0
Month_11       0
Month_12       0
DayOfWeek_1    0
DayOfWeek_2    0
DayOfWeek_3    0
DayOfWeek_4    0
DayOfWeek_5    0
DayOfWeek_6    0
Open-Close     0
High-Low       0
Low-Close      0
dtype: int64


In [240]:
import os

# Create the folder if it doesn't exist
os.makedirs("Dataset", exist_ok=True)

# Set the output file path
OUT_PATH = "./Dataset/Bitcoin-Cleaned-Dataset.csv"

# Save the DataFrame
df.to_csv(OUT_PATH, index=False)

print(f"\n✅ Dataset saved to {OUT_PATH} successfully!")


✅ Dataset saved to ./Dataset/Bitcoin-Cleaned-Dataset.csv successfully!
